In [1]:
# pygame 설치 - 17점수까지
# pip install pygame

##### [pygame 프로젝트]

1. 3월 개발 리스트 
   - 장애물 개수 추가 
   - 장애물 별 attack 수 변경
   
   
2. 4월 개발 리스트


In [2]:
"""
====================
화면 변수 설정
====================
"""
import pygame 
import random

# 게임 초기화
pygame.init()

# 화면 너비 및 게임 시간 체크
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 900

# 이미지 위치 
location_url = 'shoot_rock/'

# 화면 색상 변수
screenColor_Black = (0, 0, 0)
screenColor_Red = (255, 0, 0)
screenColor_Green = (0, 255, 0)
screenColor_Blue = (0, 0, 255)
screenColor_White = (255, 255, 255)
screenColor_Gray = (128, 128, 128)
screenColor_Yellow = (255, 255, 0)

# 폰트 변수 생성 
large_font = pygame.font.SysFont('malgungothic', 72)
small_font = pygame.font.SysFont('malgungothic', 36)

# 이미지 변수 생성 
background_image = pygame.image.load(location_url + 'planet.png')
fighter_image = pygame.image.load(location_url + 'fighter.png')
attack_image = pygame.image.load(location_url + 'missile1.png')
explosion_image = pygame.image.load(location_url + 'explosion.png')

# 장애물
wall_level_1_image = pygame.image.load(location_url + 'wall_level_1.png')
wall_level_2_image = pygame.image.load(location_url + 'wall_level_2.png')

# 음악 생성
pygame.mixer.init()
pygame.mixer.music.load(location_url + 'JLD_-_Lethal_Xcess_Level_4.mp3')
pygame.mixer.music.set_volume(0.5)
pygame.mixer.music.play(-1) # 무한 반복

attack_sound = pygame.mixer.Sound(location_url + 'missile.wav')
attack_sound.set_volume(0.3)

explosion_sound = pygame.mixer.Sound(location_url + 'explosion.wav')
explosion_sound.set_volume(0.3)

game_over_sound = pygame.mixer.Sound(location_url + 'game_over.wav')
game_over_sound.set_volume(0.3)


pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
"""
====================
공통 DEFAULT 변수 설정
====================
"""
# 기본 ZERO 값 
DEFAULT_ZERO = 0

# 화면 시메트릭
DEFAULT_CENTERX = SCREEN_WIDTH // 2
DEFAULT_CENTERY = SCREEN_HEIGHT // 2

# 비행기 이동 기본 값
DEFAULT_FIGHTER_KEY = 5

# ATTACK 기본 값 
DEFAULT_ATTACK = 1
DEFAULT_TWO_ATTACK = 2
DEFAULT_ATTACK_SPPED = 25

# MISS 기본 값 
DEFAULT_MISS = 1

# MISS 종료 기본 값 
DEFAULT_END_MISS = 4

# 랜덤 기본 값
DEFAULT_RANDOM_NUMBER = random.randint(3, 9)

# TOP DOWN 기본 값
DEFAULT_TOP_DOWN_NUMBER = -100

# 벽돌 이미지 랜덤 Y 위치 값
DEFAULT_LEVEL1_SCREEN_WIDTH = SCREEN_WIDTH - wall_level_1_image.get_width()
DEFAULT_LEVEL2_SCREEN_WIDTH = SCREEN_WIDTH - wall_level_2_image.get_width()

# 점수 기본 값 
DEFAULT_SCORE = 1

# 장애물 생성 개수 기본 값
DEFAULT_ROCKS = 3
DEFAULT_DOUBLE_ROCKS = 1

"""
====================
공통 변수 설정
====================
"""
# TOTAL 점수 
TOTAL_SCORE = 0

# DOUBLE ATTACK 기본 값
DOUBLE_ATTACK_COUNT = 0





In [4]:
"""
====================
초기 변수 (변수)
====================
"""
# 변수 및 배열 선언
fighter = fighter_image.get_rect(centerx=SCREEN_WIDTH // 2, bottom=SCREEN_HEIGHT)
rocks = []
double_rocks = []
attacks = []
missiles = []
score = 0
missed = 0
gameOver = False



# 기본 장애물 ATTACK TMP 변수 
one_attack_tmp_score = 0
one_attack_tmp_rocks = []
one_attack_tmp_attacks = []


# 더블 장애물 ATTACK TMP 변수 
double_attack_tmp_score = 0
double_attack_tmp_count = 0
double_attack_tmp_rocks = []
double_attack_tmp_attacks = []






In [5]:
"""
====================
장애물 생성 (변수)
====================
"""
# 바위 생성
for i in range(DEFAULT_ROCKS):
    rock = wall_level_1_image.get_rect(left=random.randint(0, DEFAULT_LEVEL1_SCREEN_WIDTH), top= DEFAULT_TOP_DOWN_NUMBER)
    dy = DEFAULT_RANDOM_NUMBER
    rocks.append((rock, dy))


# 더블 바위 생성 
for i in range(DEFAULT_DOUBLE_ROCKS):
    double_rock = wall_level_2_image.get_rect(left=random.randint(0, DEFAULT_LEVEL2_SCREEN_WIDTH), top= DEFAULT_TOP_DOWN_NUMBER)
    dy = DEFAULT_RANDOM_NUMBER
    double_rocks.append((double_rock, dy))
    
    

In [6]:
"""
====================
함수
====================
"""
# 기본 장애물 MISS 분기 처리 함수 
def handlingBranchMissRock(rocks, missed):
    for rock, dy in rocks:
        rock.top += dy
        if rock.top > SCREEN_HEIGHT:
            rocks.remove((rock, dy))
            rock = wall_level_1_image.get_rect(left=random.randint(DEFAULT_ZERO, DEFAULT_LEVEL1_SCREEN_WIDTH), top=DEFAULT_TOP_DOWN_NUMBER)
            dy = DEFAULT_RANDOM_NUMBER
            rocks.append((rock, dy))
            missed += DEFAULT_MISS
    return missed


# 더블 장애물 MISS 분기 처리 함수 
def handlingBranchMissDoubleRock(double_rocks, missed):
    for double_rock, dy in double_rocks:
        double_rock.top += dy
        if double_rock.top > SCREEN_HEIGHT:
            double_rocks.remove((double_rock, dy))
            double_rock = wall_level_2_image.get_rect(left=random.randint(DEFAULT_ZERO, DEFAULT_LEVEL2_SCREEN_WIDTH), top = DEFAULT_TOP_DOWN_NUMBER)
            dy = DEFAULT_RANDOM_NUMBER
            double_rocks.append((double_rock, dy))
            missed += DEFAULT_MISS            
    return missed

# 기본 장애물 Attack 분기 처리 함수 
def handlingBranchAttackRock(rocks, attacks, score):
    for rock, dy in rocks:
        for attack in attacks:
            if attack.colliderect(rock):
                rocks.remove((rock, dy))
                attacks.remove(attack)
                screen.blit(explosion_image, (rock.left, rock.top))
                rock = wall_level_1_image.get_rect(left=random.randint(DEFAULT_ZERO, DEFAULT_LEVEL1_SCREEN_WIDTH), top= DEFAULT_TOP_DOWN_NUMBER)
                dy = DEFAULT_RANDOM_NUMBER
                rocks.append((rock, dy))
                score += DEFAULT_SCORE
                explosion_sound.play()
    return score


# 더블 장애물 Attack 분기 처리 함수 
def handlingBranchAttackDoubleRock(double_rocks, attacks, score, DOUBLE_ATTACK_COUNT):
    for double_rock, dy in double_rocks:
        for attack in attacks:
            if attack.colliderect(double_rock):
                attacks.remove(attack)
                DOUBLE_ATTACK_COUNT += DEFAULT_ATTACK
                print("DOUBLE_ATTACK_COUNT : ", DOUBLE_ATTACK_COUNT)
                
                # 2번 공격했을 경우 
                if DOUBLE_ATTACK_COUNT >= DEFAULT_TWO_ATTACK:    
                    double_rocks.remove((double_rock, dy))
#                   attacks.remove(attack)
                    screen.blit(explosion_image, (double_rock.left, double_rock.top))
                    double_rock = wall_level_2_image.get_rect(left=random.randint(DEFAULT_ZERO, DEFAULT_LEVEL2_SCREEN_WIDTH), top= DEFAULT_TOP_DOWN_NUMBER)
                    dy = DEFAULT_RANDOM_NUMBER
                    double_rocks.append((double_rock, dy))
                    score += DEFAULT_SCORE
                    DOUBLE_ATTACK_COUNT = DEFAULT_ZERO
                    explosion_sound.play()
                    
    return score, DOUBLE_ATTACK_COUNT
    

# 종료 체크 함수
def handlingBranchMissGameOver(missed):
#     if missed >= DEFAULT_END_MISS:
    if missed >= DEFAULT_END_MISS:
        gameOver = True
        pygame.mixer.music.stop()
        game_over_sound.play()

    return gameOver
        
# 공격 기본 함수 
def handlingBranchDefaultAttck(attacks):
    for attack in attacks:
        attack.top -= DEFAULT_ATTACK_SPPED        # 공격속도
        if attack.top < DEFAULT_ZERO:
            attacks.remove(attack)
    
# 비행기 이동 함수
def handlingBranchMoveFighter(fighter):
    if fighter.left < DEFAULT_ZERO:
        fighter.left = DEFAULT_ZERO
    elif fighter.right > SCREEN_WIDTH:
        fighter.right = SCREEN_WIDTH
        
# 기본 장애물 접촉 gameOver 함수 
def handlingBranchColliderectRocks(rocks, fighter):
    for rock, dy in rocks:
        if rock.colliderect(fighter):
            gameOver = True
            pygame.mixer.music.stop()
            game_over_sound.play()

    return gameOver

# 기본 장애물 접촉 gameOver 함수 
def handlingBranchColliderectDoubleRocks(double_rocks, fighter):
    for double_rock, dy in double_rocks:
        if double_rock.colliderect(fighter):
            gameOver = True
            pygame.mixer.music.stop()
            game_over_sound.play()    
            
    return gameOver

In [7]:
   
# pygame.key.set_repeat(1, 1)

"""
====================
게임 루프 로직 (실제 로직)
====================
"""
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT)) 
clock = pygame.time.Clock() 

while True: 
    screen.blit(background_image, (0, 0))

    #변수 업데이트
    event = pygame.event.poll() #이벤트 처리
    if event.type == pygame.QUIT:
        break
    elif event.type == pygame.KEYDOWN:
        if event.key == pygame.K_SPACE:
            attack = pygame.Rect(attack_image.get_rect(centerx=fighter.centerx, top=fighter.top))
            attacks.append(attack)
            attack_sound.play()
        
    pressed = pygame.key.get_pressed()
    if pressed[pygame.K_LEFT] and not gameOver:
        fighter.left -= DEFAULT_FIGHTER_KEY
    elif pressed[pygame.K_RIGHT] and not gameOver:
        fighter.left += DEFAULT_FIGHTER_KEY
    
    if not gameOver:
        # 장애물 로직
        missed = handlingBranchMissRock(rocks, missed)
#         for rock, dy in rocks:
#             rock.top += dy
#             if rock.top > SCREEN_HEIGHT:
#                 rocks.remove((rock, dy))
#                 rock = wall_level_1_image.get_rect(left=random.randint(DEFAULT_ZERO, DEFAULT_LEVEL1_SCREEN_WIDTH), top = DEFAULT_TOP_DOWN_NUMBER)
#                 dy = DEFAULT_RANDOM_NUMBER
#                 rocks.append((rock, dy))
#                 missed += DEFAULT_MISS
        
        # 더블 장애물 로직 
        missed = handlingBranchMissDoubleRock(double_rocks, missed)
#         for double_rock, dy in double_rocks:
#             double_rock.top += dy
#             if double_rock.top > SCREEN_HEIGHT:
#                 double_rocks.remove((double_rock, dy))
#                 double_rock = wall_level_2_image.get_rect(left=random.randint(DEFAULT_ZERO, DEFAULT_LEVEL2_SCREEN_WIDTH), top = DEFAULT_TOP_DOWN_NUMBER)
#                 dy = DEFAULT_RANDOM_NUMBER
#                 double_rocks.append((double_rock, dy))
#                 missed += DEFAULT_MISS
        
        
        # 종료 체크 로직
        gameOver = handlingBranchMissGameOver(missed)
#         if missed >= DEFAULT_END_MISS:
#             gameOver = True
#             pygame.mixer.music.stop()
#             game_over_sound.play()
            

        # 공격 로직
        handlingBranchDefaultAttck(attacks)
#         for attack in attacks:
#             attack.top -= DEFAULT_ATTACK_SPPED        # 공격속도
#             if attack.top < DEFAULT_ZERO:
#                 attacks.remove(attack)


        # 비행기 이동 로직 
        handlingBranchMoveFighter(fighter)
#         if fighter.left < DEFAULT_ZERO:
#             fighter.left = DEFAULT_ZERO
#         elif fighter.right > SCREEN_WIDTH:
#             fighter.right = SCREEN_WIDTH

        # 장애물 격파 로직 
        one_attack_tmp_score = handlingBranchAttackRock(rocks, attacks, score)
        TOTAL_SCORE += one_attack_tmp_score
#         for rock, dy in rocks:
#             for attack in attacks:
#                 if attack.colliderect(rock):
#                     rocks.remove((rock, dy))
#                     attacks.remove(attack)
#                     screen.blit(explosion_image, (rock.left, rock.top))
#                     rock = wall_level_1_image.get_rect(left=random.randint(DEFAULT_ZERO, DEFAULT_LEVEL1_SCREEN_WIDTH), top= DEFAULT_TOP_DOWN_NUMBER)
#                     dy = DEFAULT_RANDOM_NUMBER
#                     rocks.append((rock, dy))
#                     score += DEFAULT_SCORE
#                     explosion_sound.play()

        # 더블 장애물 로직 
        double_attack_tmp_score, double_attack_tmp_count = handlingBranchAttackDoubleRock(double_rocks, attacks, score, DOUBLE_ATTACK_COUNT)
        TOTAL_SCORE += double_attack_tmp_score
        DOUBLE_ATTACK_COUNT += double_attack_tmp_count
                
#         for double_rock, dy in double_rocks:
#             for attack in attacks:
#                 if attack.colliderect(double_rock):
#                     attacks.remove(attack)
#                     double_attack_count += DEFAULT_ATTACK
                
#                     # 2번 공격했을 경우 
#                     if double_attack_count == DEFAULT_TWO_ATTACK:    
#                         double_rocks.remove((double_rock, dy))
# #                         attacks.remove(attack)
#                         screen.blit(explosion_image, (double_rock.left, double_rock.top))
#                         double_rock = wall_level_2_image.get_rect(left=random.randint(DEFAULT_ZERO, DEFAULT_LEVEL2_SCREEN_WIDTH), top= DEFAULT_TOP_DOWN_NUMBER)
#                         dy = DEFAULT_RANDOM_NUMBER
#                         double_rocks.append((double_rock, dy))
#                         score += DEFAULT_SCORE
#                         double_attack_count = DEFAULT_ZERO
#                         explosion_sound.play()
                       
        gameOver = handlingBranchColliderectRocks(rocks, fighter)
#         for rock, dy in rocks:
#             if rock.colliderect(fighter):
#                 gameOver = True
#                 pygame.mixer.music.stop()
#                 game_over_sound.play()
                
        gameOver = handlingBranchColliderectDoubleRocks(double_rocks, fighter)
#         for double_rock, dy in double_rocks:
#             if double_rock.colliderect(fighter):
#                 gameOver = True
#                 pygame.mixer.music.stop()
#                 game_over_sound.play()
   
    for rock, dy in rocks:
        screen.blit(wall_level_1_image, rock)

    for double_rock, dy in double_rocks:
        screen.blit(wall_level_2_image, double_rock)
        
    for attack in attacks:
        screen.blit(attack_image, attack)

    screen.blit(fighter_image, fighter)

    score_image = small_font.render('점수 {}'.format(TOTAL_SCORE), True, screenColor_Yellow)
    screen.blit(score_image, (10, 10))

    missed_image = small_font.render('놓친 운석 {}'.format(missed),  True, screenColor_Yellow)
    screen.blit(missed_image, missed_image.get_rect(right=SCREEN_WIDTH - 10, top=10))

    if gameOver:
        game_over_image = large_font.render('Game Over', True, screenColor_Red)
        screen.blit(game_over_image, game_over_image.get_rect(centerx=DEFAULT_CENTERX, centery= DEFAULT_CENTERY))

    pygame.display.update() 
    # FPS 프레임
    clock.tick(30) 

    
pygame.quit() 

UnboundLocalError: local variable 'gameOver' referenced before assignment